## 爬取新浪网新闻信息

In [89]:
import requests
import json
res = requests.get('http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&show_ext=1&show_all=1&show_num=500&tag=1&format=json&page=1&callback=newsloadercallback&_=1496538497721')
#res.text
jd = res.text.lstrip('  newsloadercallback(').rstrip(');')
data = json.loads(jd)

In [93]:
#data

## 建立新闻内文抓取函数

In [95]:
from bs4 import BeautifulSoup
def getArticle(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    return ' '.join([p.text.strip() for p in soup.select('#artibody p')])

#getArticle('http://news.sina.com.cn/o/2017-06-04/doc-ifyfuvpm7356166.shtml')

## 抓取新闻资料

In [21]:
ary = []
for rec in data['result']['data']:
    try:
        ary.append({'title':rec['ext3'], 'url':rec['url'], 'content': getArticle(rec['url'])})
    except:
        print(rec['ext3'])

## 储存新闻数据

In [43]:
import pandas
df = pandas.DataFrame(ary)
df.to_excel('news.xlsx')

## 读取新闻数据

In [96]:
import pandas
df = pandas.read_excel('Data/news.xlsx') 
df.head()

,content,title,url
0,英国《每日电讯报》记者Harry Yorke 称，伦敦桥附近再次传出巨大爆炸声，爆炸强度是半...,伦敦桥附近再传爆炸声,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
1,[环球网综合报道]据英国《每日邮报》6月1日报道，近日，美国洛杉矶市比弗利山庄罗迪欧大道诊所...,美诊所1.5万人整形记录被盗,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
2,[环球网综合报道]据美国有线电视新闻网4日报道，当地时间3日，英国伦敦相继发生三起袭击事件，...,特朗普“发推”声援伦敦,http://news.sina.com.cn/o/2017-06-04/doc-ifyfu...
3,原标题：军情锐评 | 文在寅为何成“挺萨德派”？俄军对付美军有一套…… 近日，美国展示拦截洲...,文在寅为何成“挺萨德派”,http://news.sina.com.cn/w/zx/2017-06-04/doc-if...
4,中新网6月4日电 据外媒报道，在伦敦桥和巴罗市场3日遭恐袭后，有目击者称，在巴罗市场连续传来...,2名疑似袭击者被击毙,http://news.sina.com.cn/o/2017-06-04/doc-ifyfu...


## 安裝Jieba

In [48]:
! pip install jieba

  Using cached jieba-0.38.zip
  Running setup.py bdist_wheel for jieba: started
  Running setup.py bdist_wheel for jieba: finished with status 'done'
  Stored in directory: C:\Users\User\AppData\Local\pip\Cache\wheels\fd\8c\07\e495d158d91e11f50c829520f77d20d1f408f18a13cdb15d02
Successfully built jieba


## 使用Jieba 断词

In [101]:
import jieba
titles   = []
articles = [] 
for rec in df.iterrows():
    #print(rec[1].content)
    articles.append(' '.join(jieba.cut(rec[1].content)))
    titles.append(rec[1].title)

In [103]:
len(articles)
len(titles)

498

In [104]:
articles[0]

'英国 《 每日 电讯报 》 记者 Harry   Yorke   称 ， 伦敦桥 附近 再次 传出 巨大 爆炸声 ， 爆炸 强度 是 半小时 前 传出 的 三起 爆炸声 的 四五倍 。   伦敦 警方 已经 证实 ， 近 45 分钟 内 发生 的 一系列 爆炸 为 “ 受控 爆破 ” （ controlled \xa0 explosions ） 。 “ 受控 爆破 ” 是 指 引爆 或者 拆除 可疑 爆炸 装置 的 一种 方法 。 （ 央视 记者 \xa0 张勇 ）   责任编辑 ： 张迪'

## 建立词频矩阵

In [121]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(articles)

In [122]:
X.shape

(498, 16527)

## 计算余弦距离(Cosine Similarity)

In [123]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities  = cosine_similarity(X, X)

In [127]:
cosine_similarities.shape
cosine_similarities

array([[ 1.        ,  0.03107505,  0.14547859, ...,  0.05066481,
         0.1387863 ,  0.01562691],
       [ 0.03107505,  1.        ,  0.13350354, ...,  0.07793338,
         0.14340031,  0.02294493],
       [ 0.14547859,  0.13350354,  1.        , ...,  0.19071546,
         0.29149942,  0.01790287],
       ..., 
       [ 0.05066481,  0.07793338,  0.19071546, ...,  1.        ,
         0.15117901,  0.08550731],
       [ 0.1387863 ,  0.14340031,  0.29149942, ...,  0.15117901,
         1.        ,  0.03795401],
       [ 0.01562691,  0.02294493,  0.01790287, ...,  0.08550731,
         0.03795401,  1.        ]])

## 使用KMeans 分群

In [128]:
from sklearn.cluster import KMeans
c = KMeans(n_clusters=10, init = 'k-means++', random_state=123)
k_data = c.fit_predict(cosine_similarities)

In [129]:
k_data

array([4, 4, 0, 9, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 7, 3, 4, 2, 5, 4, 3, 3,
       5, 4, 6, 4, 4, 4, 4, 2, 0, 3, 1, 2, 3, 4, 4, 4, 4, 4, 5, 4, 3, 4, 3,
       4, 7, 3, 3, 4, 4, 2, 3, 4, 4, 7, 4, 4, 2, 9, 2, 1, 4, 3, 8, 4, 7, 4,
       9, 2, 7, 2, 7, 4, 4, 5, 7, 4, 4, 2, 2, 2, 7, 7, 3, 4, 4, 8, 7, 3, 4,
       2, 5, 2, 4, 3, 7, 2, 9, 7, 5, 5, 2, 3, 4, 3, 4, 2, 3, 8, 7, 3, 4, 8,
       2, 0, 2, 2, 2, 3, 9, 4, 3, 8, 4, 4, 4, 3, 3, 7, 4, 4, 5, 4, 2, 3, 4,
       5, 5, 3, 5, 0, 5, 3, 4, 5, 4, 2, 2, 3, 4, 2, 4, 3, 6, 0, 4, 0, 7, 4,
       4, 3, 4, 3, 4, 3, 8, 4, 5, 8, 4, 2, 4, 4, 4, 4, 4, 3, 9, 4, 7, 3, 4,
       2, 4, 2, 9, 4, 4, 3, 9, 3, 4, 4, 8, 4, 0, 4, 4, 3, 9, 4, 8, 8, 2, 8,
       4, 2, 3, 4, 4, 5, 4, 6, 4, 4, 1, 5, 9, 4, 3, 5, 2, 9, 9, 8, 4, 4, 7,
       3, 5, 4, 4, 4, 5, 6, 4, 0, 3, 2, 9, 2, 3, 4, 8, 8, 4, 6, 4, 6, 9, 1,
       8, 3, 6, 6, 9, 1, 6, 1, 8, 1, 6, 6, 4, 8, 6, 4, 8, 8, 3, 4, 6, 4, 4,
       6, 6, 5, 4, 6, 4, 4, 5, 4, 8, 7, 8, 0, 5, 9, 3, 8, 6, 4, 4, 3, 0, 1,
       1, 9,

## 产生分群结果

In [131]:
#k_data == 8

In [139]:
import numpy
titles_ary = numpy.array(titles)
titles_ary[k_data == 9]

array(['文在寅为何成“挺萨德派”', '美韩同盟裂痕扩大？', '环评分歧或影响萨德部署', '韩强调将落实萨德环评工作',
       '韩方漏报萨德意使部署不可逆', '韩防长回应瞒报“萨德”入韩', '韩国女子美使馆前反萨德', '韩青瓦台通知前国防部长受查',
       '文在寅:不会改变部署决定', '文在寅:不改变萨德部署决定', '韩媒：韩军或遭大换血', '萨德瞒报丑闻指向韩防长',
       '文在寅:不会改变萨德部署决定', '韩防长:未曾下令瞒报萨德入韩', '美方:韩萨德仅有初步拦截能力',
       '韩方回应萨德发射车暗中入韩', '韩军瞒报4辆萨德车秘密入境', '萨德发射架新政府成立前入境', '4套萨德发射车“秘密”入韩',
       '美媒:萨德成文在寅手中悬案', '外媒:萨德成文在寅烫手山芋'], 
      dtype='<U16')